In [1]:
import pandas as pd
from sqlalchemy import create_engine



# Extract CSVs into DataFrames

In [3]:
election2016_file = "../Resources/elections_2016.csv"
election2016_df = pd.read_csv(election2016_file)
election2016_df.head()

,State,Trump (R),Clinton (D),All Others,Total Vote
0,AL,"1,318,255","729,547","75,570","2,123,372"
1,AK,"163,387","116,454","38,767","318,608"
2,AZ,"1,252,401","1,161,167","159,597","2,573,165"
3,AR,"684,872","380,494","65,310","1,130,676"
4,CA,"4,483,814","8,753,792","943,998","14,181,604"


In [4]:
gun_ownership_2016 = "../Resources/gun_ownership.csv"
gun_ownership_2016_df = pd.read_csv(gun_ownership_2016)
gun_ownership_2016_df.head()

,State,# of guns per capita,# of guns registered
0,Wyoming,229.24,"132,806"
1,Washington,68.05,"47,228"
2,New Hampshire,46.76,"64,135"
3,New Mexico,46.73,"97,580"
4,Virginia,36.34,"307,822"


In [6]:
# to add abrivitation of states
states_abbreviation = "../Resources/states.csv"
states_df = pd.read_csv(states_abbreviation)
states_df.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [7]:

merged_data = gun_ownership_2016_df.merge(states_df, on='State')
merged_data.head()

,State,# of guns per capita,# of guns registered,Abbreviation
0,Wyoming,229.24,"132,806",WY
1,Washington,68.05,"47,228",WA
2,Washington,12.40,"91,835",WA
3,New Hampshire,46.76,"64,135",NH
4,New Mexico,46.73,"97,580",NM


In [8]:
new_df_gun = merged_data.drop(['State'], axis = 1) 
new_df_gun.head()

,# of guns per capita,# of guns registered,Abbreviation
0,229.24,"132,806",WY
1,68.05,"47,228",WA
2,12.40,"91,835",WA
3,46.76,"64,135",NH
4,46.73,"97,580",NM


### Transform elections DataFrame

In [9]:
election2016_df.columns

Index(['State ', 'Trump (R)', 'Clinton (D)', 'All Others', 'Total Vote'], dtype='object')

In [10]:
# Create a filtered dataframe from specific columns
election_cols = ["State ", "Trump (R)", "Clinton (D)", "All Others"]
election_transformed = election2016_df[election_cols].copy()

# Rename the column headers
election_transformed = election_transformed.rename(columns={'State ':"State", 'Trump (R)':"Trump", 'Clinton (D)':"Clinton"})

# Set index
election_transformed.set_index("State", inplace=True)

election_transformed.head()

,Trump,Clinton,All Others
State,,,
AL,"1,318,255","729,547","75,570"
AK,"163,387","116,454","38,767"
AZ,"1,252,401","1,161,167","159,597"
AR,"684,872","380,494","65,310"
CA,"4,483,814","8,753,792","943,998"


# Transform gun owned DataFrame

In [11]:
new_df_gun.columns

Index(['# of guns per capita', '# of guns registered', 'Abbreviation'], dtype='object')

In [12]:
# Create a filtered dataframe from specific columns
gun_cols = ["# of guns per capita" , "# of guns registered","Abbreviation"]
gun_transformed = new_df_gun[gun_cols].copy()

# Rename the column headers
gun_transformed = gun_transformed.rename(columns={'# of guns per capita':"num_guns_per_cap", '# of guns registered':"num_guns_regist", 'Abbreviation':"State"})

# Set index
gun_transformed.set_index("State", inplace=True)

gun_transformed.head()

,num_guns_per_cap,num_guns_regist
State,,
WY,229.24,"132,806"
WA,68.05,"47,228"
WA,12.40,"91,835"
NH,46.76,"64,135"
NM,46.73,"97,580"


# Create database connection

In [16]:
connection_string = "postgres:Meltem260584@localhost:5432/gun_election_relation"
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
# Confirm tables
engine.table_names()

[]

# Load DataFrames into database

In [18]:
election_transformed.to_sql(name='elections_2016', con=engine, if_exists='append', index=True)

In [19]:
gun_transformed.to_sql(name='gun_numbers', con=engine, if_exists='append', index=True)